# ДЗ №1 - обучение модели линейной регресии методом градиентного спуска

Реализовать обучение модели "нейросети" (в варианте линейной регрессии) методом градиентного спуска.<br />

В качестве подводящего упражнения в этом задании предлагается реализовать функции потерь и саму модель "нейросети" (линейной регрессии) в манере, схожей с построением модулей фреймворка pytorch (см. пояснения в шаблонах кода)

В решении ожидается наличие следующих ключевых составляющих:<br />

#### Текстовое описание в решении:
- формулировка задачи, формулировка признакового описания объектов, формулировка функции ошибки;
- исследование исходных данных на предмет скоррелированности признаков; фильтрация признаков; порождение признаков (при необходимости);
- оценка параметров модели линейной регрессии (обучение модели) методом градиентного спуска;

#### Код решения:
(используйте предлагаемые шаблоны)
- формулировка всех составляющих модели "нейросети";
- формулировка модели "нейросети" - `NN` (линейной регрессии);
- формулировка функции ошибки вместе с ее составляющими (например, класс отклонения `Residual`);
- формулировка цикла оптимизации параметров.


#### Визуализация в решении:
- распределения признаков;
- распределение целевой переменной;
- эволюция функции ошибки и выбранных метрик качества по ходу обучения.
- диаграмма соответствия измеренной целевой переменной и значений целевой переменной, оцененной с использованием обученной "нейросети"

#### Выводы (в форме текста!)
- вывод о том, насколько модель подходит для описания данных
- вывод о достаточности или избыточности данных для оценки параметров модели
- вывод о соотношении выразительности модели и ее обобщающей способности (наблюдаются ли явления переобучения или недообучения).

Примечания:<br />
Допустимо порождение признаков (полиномиальных, экспоненциальных, логарифмических, etc.)<br />
Реализация линейной регрессии может быть написана только с использованием библиотеки Numpy. Решения с использованием библиотек автоматического вычисления градиентов не засчитываются.<br />
Из готовых реализаций (напр., из пакета scikit-learn) в этом задании допускается использовать только порождение полиномиальных признаков `PolynomialFeatures`.

Данные находятся в следующих файлах:

Признаковое описание объектов обучающей выборки - в файле X_train.npy

Значения целевой переменной на обучающей выборке - в файле y_train.npy

Способ чтения данных из файлов *.npy :

```
import numpy as np
data = np.load('/path/to/filename.npy')
```

### Примечание на предмет реализации градиента функции потерь

Нелишним будет вспомнить способ вычисления градиента сложной функции. Здесь функция ошибки (обозначено как $\mathscr{L}$) представлена как сложная функция $\mathscr{L}\left( G\left( \theta \right) \right)$. Для простоты приведена сразу матричная запись. По ней можно сверить свой результат.

$$
\nabla_{\theta}{\mathscr{L}} = \nabla_{G}{\mathscr{L}}\cdot\nabla_{\theta}{G}
$$

В качестве шпаргалки можно подсмотреть правила матричного дифференцирования <a href="https://www.math.uwaterloo.ca/~hwolkowi/matrixcookbook.pdf">здесь</a>

Например, в случае функции потерь MSE это может выглядеть следующим образом:

$$
\nabla_{\theta}{\mathscr{L}} = 2\left(X\theta - Y\right) \cdot \mathbf{I} \cdot X
$$

В этом ДЗ следует очень аккуратно реализовать градиент каждой отдельной операции по аргументу этой операции:
- градиент квадрата отклонения $d^2$ - по отклонению $d$
- градиент отклонения $\left(\hat{y}-y\right)$ - по аргументу $\hat{y}$
- градиент функции активации $\hat{y}=A\left(z\right)$ - по ее агрументу $z$ (да, здесь функция активации $A$ - Identity, но это сейчас не так важно)
- градиент оценки $z=\theta^T\cdot X$ - по аргументу $\theta$

**ВНИМАНИЕ**
В этом задании также следует учесть, что подразумевается, что метод `backward` каждого класса выдает градиент **функции ошибки** по аргументу операции. Для учета градиентов всех предыдущих операций в этот метод передается т.н. **upstream gradient** - переменная `usg`. Не забывайте ее передавать при "сборке" полного градиента. Эта "сборка" у вас будет в двух классах - классе функции потерь `MSE` (нужно собрать градиент операции `MSE` с учетом того, что она, в свою очередь, сложная функция, использующая `Residual`) и классе нейросети `NN`.

Как можно видеть, все операции, из которых составляется "нейросеть" в этом задании, могут быть представлены однотипно: для всех из них можно задать метод вычисления `forward` на "прямом проходе" и метод вычисления градиента `backward` на этапе вычисления градиентов, "обратном проходе".

**ВНИМАНИЕ**
Не следует забывать, что для вычисления градиентов обычно используются результаты операции, вычисленные на этапе "прямого прохода". Для хранения этих результатов используйте атрибуты класса `cache`. Напомним, ссылка на сам экземпляр класса в теле метода класса обычно упоминается как `self`. То есть, атрибут `cache` этого экземпляра класса будет в этом методе упоминаться как `self.cache`. Вы можете назвать его как угодно (не обязательно именно `cache`), но реализация хранения промежуточных результатов вычисления нейросети - **обязательно** в этом ДЗ.

In [3]:
import numpy as np
from tqdm import tqdm

In [4]:
%matplotlib inline

In [5]:
import matplotlib.pyplot as plt

In [6]:
Xtr = np.load('./X_train.npy')
ytr = np.load('./y_train.npy')

In [7]:
ytr.shape

(10000, 1)

In [7]:
Xtr.shape

(10000, 4)

In [ ]:
plt.scatter(Xtr[:,0], ytr, s=1)

In [ ]:
plt.scatter(Xtr[:,1], ytr, s=1)

In [ ]:
plt.scatter(Xtr[:,2], ytr, s=1)

In [ ]:
plt.scatter(Xtr[:,3], ytr, s=1)

In [ ]:
np.corrcoef(Xtr, rowvar=False)

In [ ]:
# Примите решение о фильтрации признаков или порождении новых признаков
# Xtr = ...

In [15]:
class Differentiable:
    def __init__(self):
        pass
    
    def forward(self, **kwargs):
        raise NotImplementedError()
    
    def backward(self, **kwargs):
        raise NotImplementedError()

In [ ]:
class Residual(Differentiable):
    def __init__(self):
        super(Residual, self).__init__()
    
    def __call__(self, mu, y):
        return self.forward(mu, y)
    
    def forward(self, mu, y):
        # Этот метод реализует вычисление отклонения mu-y
        d = None
        self.cache = None
        
        return d
    
    def backward(self, usg):
        # Этот метод реализует вычисление градиента отклонения D по аргументу mu
        
        partial_grad = None
        
        ### YOUR CODE HERE
        # partial_grad = ...
        
        return partial_grad

In [16]:
class MSE(Differentiable):
    def __init__(self):
        super(MSE, self).__init__()
        self.diff = Residual()
    
    def __call__(self, mu, y):
        # d = ...
        return self.forward(d)
    
    def forward(self, d):
        # Этот метод реализует вычисление значения функции потерь
        # Подсказка: метод должен возвращать единственный скаляр - значение функции потерь
        self.cache = None
        loss_value = None
        
        return loss_value
    
    
    def backward(self):
        # Этот метод реализует вычисление градиента функции потерь по аргументу d
        # Подсказка: метод должен возвращать вектор градиента функции потерь
        #           размерностью, совпадающей с размерностью аргумента d
        
        partial_grad = None
        
        ### YOUR CODE HERE
        # partial_grad = ...
        
        return partial_grad

In [ ]:
class linear(Differentiable):
    def __init__(self):
        super(linear, self).__init__()
        self.theta = None
        self.cache = None
    
    def __call__(self, X):
        # этот метод предназначен для вычисления значения целевой переменной
        return self.forward(X)
    
    def forward(self, X):
        # этот метод предназначен для применения модели к данным
        assert X.ndim == 2, "X should be 2-dimensional: (N of objects, n of features)"
        
        # ВНИМАНИЕ! Матрица объекты-признаки X не включает смещение
        #           Вектор единиц для применения смещения нужно присоединить самостоятельно!
        
        ### YOUR CODE HERE
        # X_ = ...
        
        if (self.theta is None):
            # Если вектор параметров еще не инициализирован, его следует инициализировать
            # Подсказка: длина вектора параметров может быть получена из размера матрицы X
            # Fx1.T dot NxF.T = 1xN
            # Если X - матрица объекты-признаки, то это матрица из вектор-строк!
            self.theta = None
        
        
        # Здесь следует собственно применить модель к входным данным
        
        z = None
        self.cache = None
        
        ### YOUR CODE HERE
        # mu = ...
        # self.cache = ...
        
        return z
    
    def backward(self, usg):
        # Этот метод реализует вычисление компоненты градиента функции потерь
        
        assert self.cache is not None, "please perform forward pass first"
        
        partial_grad = None
        self.cache = None
        
        ### YOUR CODE HERE
        # partial_grad = ...
        
        # Не забудьте очистить кэш!
        # self.cache = ...
        
        return partial_grad

In [27]:
class Identity(Differentiable):
    def __init__(self):
        super(Identity, self).__init__()
    
    def __call__(self, X):
        # этот метод предназначен для вычисления значения функции активации
        return self.forward(X)
    
    def backward(self, usg):
        # Этот метод реализует вычисление компоненты градиента функции потерь
        return usg
    
    def forward(self, X):
        # этот метод предназначен для вычисления функции активации
        return X

In [28]:
class NN(Differentiable):
    def __init__(self):
        super(NN, self).__init__()
        self.l1 = linear()
        self.act = Identity()
    
    def __call__(self, X):
        return self.forward(X)
    
    def forward(self, X):
        # Этот метод будет вычислять нейросеть на данных X
        ### YOUR CODE HERE
        # x = ...
        return x
    
    def backward(self, usg):
        grad = None
        ### YOUR CODE HERE
        # grad = ...
        return grad

In [29]:
def clip_by_norm(grad, max_norm = 1.0):
    grad_norm = np.linalg.norm(grad)
    if grad_norm > max_norm:
        grad = max_norm * grad / grad_norm
    return grad

In [ ]:
network = NN()

In [ ]:
mu = network(Xtr)

In [ ]:
loss_fn = MSE()

In [ ]:
loss = loss_fn(mu, ytr)

In [ ]:
loss_fn.backward()

In [ ]:
network.backward(loss_fn.backward())

In [ ]:
learning_rate = 1e-4
epochs = 10000

### Далее идет процедура обучения созданной нейросети

In [ ]:
loss_history = []
pbar = tqdm(total=epochs)
for epoch in range(epochs):
    mu = None
    loss_value = None
    grad = None
    grad = clip_by_norm(grad, 1.0)
    
    ### YOUR CODE HERE
    # mu = ...
    # loss_value = ...
    # grad = ...
    # grad = clip_by_norm(grad, 10)
    
    # update network parameters
    # network.l1.theta = ... + ...
    loss_history.append(loss_value)
    pbar.update(1)
    pbar.set_postfix({'loss': loss_value})
pbar.close()

In [ ]:
# отобразите эволюцию функции потерь по мере обучения сети
plt.plot(loss_history)
plt.yscale('log')

In [ ]:
# примените нейросеть к данным Xtr
mu = network(Xtr)

In [ ]:
# отобразите диаграмму y(y_true) для оценки соответствия полученного решения известному
plt.scatter(ytr, mu, s=1)